In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-06-18 15:43:48--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-18 15:43:48 (33.9 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [10]:
import minsearch

In [2]:
import json

In [3]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [4]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [5]:
# we have built documents from docs_raw, eliminating the nested stucture. 
# Now "course" is at the same level as "text", "section" and "question" for any document.
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [14]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [3]:
q = 'the course has already started, can I still enroll?'

In [15]:
index.fit(documents)

In [9]:
boost = {'question': 3.0, 'section': 0.5}

results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5   
)

In [10]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [6]:
#introduce LLMs (requires opeai api key setup. I have made use of codespaces secrets)
from openai import OpenAI

In [7]:
client = OpenAI()

In [4]:
#let's query the LLM without giving any context from our knowledgebase
#as you can see, you will receive a generic response

response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"It's common to be able to enroll in a course after it has already started, but it depends on the specific policies of the institution offering the course. Here are a few steps you can take to find out:\n\n1. **Check the Course Website**: Look for any information about late enrollment policies. Some courses may have specific deadlines or allow late enrollment under certain conditions.\n\n2. **Contact the Instructor or Course Coordinator**: Send an email or visit during office hours to inquire about the possibility of joining the course late. Be sure to explain why you want to enroll and show that you're committed to catching up on any missed material.\n\n3. **Speak with an Academic Advisor**: Academic advisors can provide guidance and may be able to help you with the enrollment process or offer alternatives if late enrollment is not possible.\n\n4. **Registrar’s Office**: The registrar’s office at the institution can often provide detailed information about enrollment deadlines and pro

In [5]:
#Let's now define some useful functions to make use of our search engine 
#to define the context of the prompt for the openai LLM

def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results


In [15]:
#.strip() used in prompt_template to avoid line breaks between different texts in the context

def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [16]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [8]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [16]:
rag(q)

'Yes, you can still enroll even after the course has started. You are eligible to submit the homeworks, but be mindful of the deadlines for turning in the final projects to avoid leaving everything for the last minute.'

In [17]:
#another example for the RAG

query = 'how do I run kafka?'

In [18]:
rag(query)

'To run Kafka, you can follow the instructions based on the type of implementation you are working on:\n\n1. **Java Kafka Producer/Consumer/KStreams**:\n   - Navigate to your project directory.\n   - Run the following command in your terminal:\n     ```\n     java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n     ```\n\n2. **Python Kafka**:\n   - If encountering a "Permission denied" error, ensure you grant execute permission to the build script:\n     ```\n     chmod +x build.sh\n     ```\n   - If you find a "ModuleNotFoundError," it is recommended to use the updated Kafka Python package:\n     ```\n     pip install kafka-python-ng\n     ```\n\n3. **Running a Python File in a Virtual Environment**:\n   - Create a virtual environment and install the required packages:\n     ```\n     python -m venv env\n     source env/bin/activate\n     pip install -r ../requirements.txt\n     ```\n   - Activate the virtual environment every time you need

In [8]:
#let's change the search engine from minsearch to elasticsearch
#before doing that it is necessary to launch the container elastisearch in the bash terminal..

from elasticsearch import Elasticsearch

In [9]:
es_client = Elasticsearch('http://localhost:9200')

In [11]:
#having already created the index over documents in workshop1
#we can simply recall the index by name
index_name = "course-questions"

In [18]:
query = 'I just discovered the course. Can I still join it?'

In [13]:
#let's define the search function that takes place of the one based on minsearch()

def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [14]:
def rag(query):
    search_results = elastic_search(query)  #here the only change to the rag() function
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [19]:
rag(query)

"Yes, you can still join the course even if it has already started. You are eligible to submit the homeworks regardless of your registration status. However, please keep in mind that there are deadlines for turning in the final projects, so it's important not to leave everything for the last minute."